# Lab 2c: Clasificación Multilabel

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FCEIA-AAII/lab2/blob/main/lab2c.ipynb)

## Preparación del entorno.

Si no estamos parados en el repo, clonar y cd al repo. Esto nos permite usar el mismo notebook tanto local como en Google Colab.

In [ ]:
import os

REPO_NAME = "lab2"
if REPO_NAME not in os.getcwd():
  if not os.path.exists(REPO_NAME):
    !git clone https://github.com/FCEIA-AAII/{REPO_NAME}.git
  os.chdir(REPO_NAME)



## Análisis Exploratorio.

Importar librerías

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

Establecer GPU por defecto en caso de estar disponible

In [ ]:
# Configurar para que TensorFlow utilice la GPU por defecto
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Configurar para que TensorFlow asigne memoria dinámicamente
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        # Especificar la GPU por defecto
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Manejar error
        print(e)

Cargar y visualizar los datos.

In [ ]:
data = pd.read_csv('dataset-lab2-c.csv')
print(data.head())

Plot de los datos.

In [ ]:
X1 = data['X1'].to_numpy()
X2 = data['X2'].to_numpy()
Y = data['Y'].to_numpy()

print("X1 shape:", X1.shape)
print("X2 shape:", X2.shape)
print("Y shape:", Y.shape)

plt.scatter(X1[Y == 0], X2[Y == 0], color='red', alpha=0.5)
plt.scatter(X1[Y == 1], X2[Y == 1], color='green', alpha=0.5)
plt.scatter(X1[Y == 2], X2[Y == 2], color='blue', alpha=0.5)

plt.xlabel('Promedio Parciales')
plt.ylabel('Promedio TPs')

plt.show()

Definimos nuestro modelo usando tensorflow.

In [ ]:
model = tf.keras.Sequential([
    ##### DEFINIR EL MODELO AQUI #####
    # Definir un modelo sin capas ocultas.
    # Elegir correctamente la función de activación y 
    # la cantidad de neuronas de la capa de salida en
    # base al problema.
    ##################################
])

Entrenamos el modelo.

*Nota: "sparse_categorical_crossentropy" encodea variables categoricas a one-hot y luego calcula binary_cross_entropy para cada componente. Nos permite utilizar `Y` directamente en formato {0,1,2} sin necesidad de encodearlo en one-hot.*

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.5)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

x = np.column_stack([X1, X2])
y = Y
print(x.shape)
print(y.shape)
model.fit(x=x, y=y, epochs=50)

Definimos una función para plotear la frontera de decisión sobre los datos.

In [ ]:
def plot_results(X1, X2, Y, model):
    plt.scatter(X1[Y == 0], X2[Y == 0], color='red', alpha=0.5)
    plt.scatter(X1[Y == 1], X2[Y == 1], color='green', alpha=0.5)
    plt.scatter(X1[Y == 2], X2[Y == 2], color='blue', alpha=0.5)

    plt.xlabel('Promedio Parciales')
    plt.ylabel('Promedio TPs')

    x1 = np.linspace(0, 1, 100)
    x2 = np.linspace(0, 1, 100)

    # Create a meshgrid with all the possible combinations of x1 and x2
    x1, x2 = np.meshgrid(x1, x2, indexing='ij')

    # This is equivalent to
    # x = np.array([[i, j] for i in x1 for j in x2])
    x = np.array([x1.ravel(), x2.ravel()]).T

    # Predict the value for each point in the meshgrid
    y = model.predict(x)

    # Convert from one-hot encoding to a single number
    y = np.argmax(y, axis=-1).reshape(x1.shape)

    # Use cmap red and green
    plt.contourf(x1, x2, y, alpha=0.1, cmap='RdYlGn')

    # Draw the decision boundary
    plt.contour(x1, x2, y, levels=[0, 1, 2], colors='blue')

    plt.xlim(0, 1)
    plt.ylim(0, 1)

    plt.show()

Plot de la frontera de decisión.

In [ ]:
plot_results(X1, X2, Y, model)

Vemos que el modelo llega a una solución aceptable.

Probemos entonces con un modelo más complejo (para ver que pasa).

*Nota: el entrenamiento puede no converger a una solución óptima. Se propone correr el entrenamiento varias veces hasta que se obtenga una solución aceptable. Más adelante estudiaremos técnicas para mejorar la convergencia.*

In [ ]:
model = tf.keras.Sequential([
    ##### DEFINIR EL MODELO AQUI #####
    # Definir un modelo más complejo con capas ocultas.
    # Experimentar con la cantidad de capas, neuronas y
    # funciones de activación.
    # Elegir correctamente la función de activación y 
    # la cantidad de neuronas de la capa de salida en
    # base al problema.
    ##################################
])

# Experimentar con distintos valores de learning rate
# y observar cómo afecta a la convergencia del modelo.
# Probar distintos órdenes de magnitud. Ej: 5, 0.5, 0.05, 0.005, 0.0005
optimizer = tf.keras.optimizers.Adam(learning_rate=0.5)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(x=x, y=y, epochs=50)

plot_results(X1, X2, Y, model)